### IMPORTS

In [1]:
import pandas as pd
import numpy as np
import pandas_gbq
import datetime
from dateutil.relativedelta import relativedelta
from gspread_pandas import Spread, conf

### CREDENCIALES

In [2]:
cred = conf.get_config('C:\\Users\\santiago.curat\\Pandas\\PEYA', 'PedidosYa-8b8c4d19f61c.json')

### QUERIES

In [56]:
q_adr = '''WITH barter_table AS (
    SELECT bt.trade.order_id AS order_id,
           bc.campaign_title AS campaign,
           bt.trade.amount_used AS amount_used,
           bc.discount.type AS discount_type,
           bc.discount.amount AS discount_amount,
           bc.discount.max_amount AS discount_max_amount,
           bc.rules.min_amount AS min_amount
    FROM `peya-data-origins-pro.cl_barter.trades` AS bt
    INNER JOIN `peya-data-origins-pro.cl_barter.creates` AS bc ON bt.id = bc.id
    WHERE DATE(bt.tradedAt) BETWEEN DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -3 MONTH),MONTH) AND CURRENT_DATE()
          AND DATE(bc.created_at) BETWEEN DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -3 MONTH),MONTH) AND CURRENT_DATE()),
    orders_table AS (
    SELECT o.order_id AS order_id,
           c.city_name AS city,
           SUM(IFNULL(od.discount_amount,0)) AS total_disc,
           SUM(CASE WHEN UPPER(od.discount_type_name) = 'VOUCHER' THEN IFNULL(od.discount_amount,0) ELSE 0 END) AS total_voucher_disc
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o,
    UNNEST(discounts) AS od
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` AS c ON o.city.city_id = c.city_id
    WHERE o.registered_date BETWEEN DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -2 MONTH),MONTH) AND CURRENT_DATE()
          AND o.order_status = 'CONFIRMED'
    GROUP BY 1,2),
    acq_table AS (
    SELECT o.user.id AS user,
           cn.country_name AS country,
           cn.currency_id AS currency_id,
           IFNULL(ot.city,'-') AS city,
           o.registered_date AS fecha,
           o.shipping_amount_no_discount + o.amount_no_discount AS total_paid,
           o.order_id AS order_id,
           ot.total_disc AS total_discount,
           ot.total_voucher_disc AS total_voucher_disc,
           bt.campaign AS campaign,
           bt.amount_used AS amount_used,
           bt.discount_type AS discount_type,
           bt.discount_amount AS discount_amount,
           bt.discount_max_amount AS discount_max_amount,
           bt.min_amount AS min_amount
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    LEFT JOIN barter_table AS bt ON o.order_id = bt.order_id
    LEFT JOIN orders_table AS ot ON o.order_id = ot.order_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
    WHERE o.registered_date BETWEEN DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -2 MONTH),MONTH) AND CURRENT_DATE()
          AND o.order_status = 'CONFIRMED'),
    acq_final_table AS (
    SELECT acq.user AS user,
           acq.country AS country,
           acq.currency_id AS currency_id,
           acq.city AS city,
           acq.fecha AS fecha,
           acq.campaign AS campaign,
           acq.discount_type AS discount_type,
           acq.discount_amount AS discount_amount,
           acq.discount_max_amount AS discount_max_amount,
           acq.min_amount AS min_amount,
           SUM(o.total_paid) AS total_paid,
           SUM(o.total_discount) AS total_disc,
           SUM(o.total_voucher_disc) AS total_voucher_disc,
           SUM(o.amount_used) AS amount_used,
           COUNT(DISTINCT o.order_id) AS orders
    FROM acq_table AS acq
    LEFT JOIN acq_table AS o ON acq.user = o.user
    WHERE o.fecha >= DATE_ADD(acq.fecha, INTERVAL 1 DAY)
          AND acq.campaign IS NOT NULL
    GROUP BY 1,2,3,4,5,6,7,8,9,10)
SELECT FORMAT_DATE('%Y-%m',aft.fecha) AS Month,
       aft.country AS Country,
       aft.city AS City,
       aft.campaign AS Campaign,
       aft.discount_type AS Discount_Type,
       aft.discount_amount AS Amount,
       aft.discount_max_amount AS Max_Amount,
       aft.min_amount AS MOV,
       COUNT(DISTINCT aft.user) AS Users,
       SUM(IFNULL(aft.total_paid,0)) AS Total_Paid,
       SUM(IFNULL(aft.total_disc,0)) AS Total_Disc,
       SUM(IFNULL(aft.total_voucher_disc,0)) AS Total_Voucher_Disc,
       SUM(IFNULL(aft.amount_used,0)) AS Total_Amount_Used,
       SUM(IFNULL(aft.total_paid / ce.rate_us,0)) AS Total_Paid_USD,
       SUM(IFNULL(aft.total_disc / ce.rate_us,0)) AS Total_Disc_USD,
       SUM(IFNULL(aft.total_voucher_disc / ce.rate_us,0)) AS Total_Voucher_Disc_USD,
       SUM(IFNULL(aft.amount_used / ce.rate_us,0)) AS Total_Amount_Used_USD,
       SUM(IFNULL(aft.orders,0)) AS Orders
FROM acq_final_table AS aft
LEFT JOIN `peya-bi-tools-pro.il_core.dim_currency_exchange` AS ce ON aft.currency_id = ce.currency_id AND DATE_TRUNC(aft.fecha,MONTH) = ce.currency_exchange_date
WHERE aft.campaign LIKE '%%CENTRAL%%'
GROUP BY 1,2,3,4,5,6,7,8'''

In [57]:
# Descargo la data
bq_adr = pd.io.gbq.read_gbq(q_adr, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 56176/56176 [00:24<00:00, 2251.95rows/s]


In [58]:
# Copio la data
adr = bq_adr.copy()

### TRABAJO

In [59]:
# Doy formato a las columnas
cols_str = ['Campaign','Country','Month','City','Discount_Type']
cols_float = [i for i in adr.columns if i not in cols_str]
adr[cols_float] = adr[cols_float].astype(float)
adr[cols_str] = adr[cols_str].apply(lambda x: x.astype(str).str.upper())
# Saco los espacios de los nombres de campañas
adr['Campaign'] = adr['Campaign'].str.replace(' ', '')
# Doy formato a la fecha
adr['Month'] = pd.to_datetime(adr['Month'], format='%Y-%m').dt.strftime('%Y-%m')
# Hago un listado de los paises
countries = list(adr['Country'].unique())

In [60]:
# Creo las columnas faltantes
adr['ADR_Total_Disc'] = adr['Total_Disc'] / adr['Total_Paid']
adr['ADR_Voucher_Disc'] = adr['Total_Voucher_Disc'] / adr['Total_Paid']
adr['ADR_CRM_Used'] = adr['Total_Amount_Used'] / adr['Total_Paid']
adr.replace([np.nan,np.inf,-np.inf],0,inplace=True)